# 시계열 분석
* 시계열 데이터 : 주가, 환율, 거래량, 판매량, 재고량, 수요량, 클릭률, 기온, 습도, 인구, 출생률, 트래픽양의 피크시간 패턴, 제품의 판매주기, 심장 박동
* 시계열 데이터의 특성을 파악
    * 규칙적 : 경향(trend), 계절성(seasonality), 주기(cycle)
    * 불규칙적 : 불규칙성(irregular, random)


* 추세 파악 
: 무작위 적인 소음을 제거하여 흐름을 파악
* 원인 예측 및 대응
: 수요 분석을 통한 재고량 관리
* 전망
: 영업 전략, 생산 계획




### 정상 시계열
 * 뚜렷한 추세가 없음
 * 진폭이 흐름에 따라 일정함

### 비정상 시계열
 * 평균이 시간대에 따라 다름
 * 추세, 계절성을 가짐
 * 분산이 변한다
 * 분산이 일정하지 않으면 특정 기간에 오류가 발생하고 예측을 하기에 적합하지 않을 수 있다.


### 비정상 시계열의 정상화
1. 분산이 일정하지 않은 경우 
 * 분산 안정화 변환(로그변환, 제곱근 변환, Box-Cox 변환)
2. 추세가 있을 때
 * 결정적 추세 : 분해법 또는 추세항 모형에 포함
 * 확률적 추세(Dickey-Fuller의 단위근 검정): 차분을 이용
3. 계절성을 가지는 경우
 * 결정적 계절추세 : 계절 추세항 모형에 포함
 * 확률적 계절추세(계절형 단위근 검정) : 계절차분 


## 여의나루역(대여소 번호 207)에서 대여하거나 반납한 자전거의 이력을 분석
* 여의나루역은 같은 대여소에서 대여반납이 가장 많은 지역
* 대여 혹은 반납이 여의나루역인 데이터
* 2017년 10월부터 2019년 5월까지의 데이터를 사용

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='AppleGothic') 
plt.rc('axes', unicode_minus=False)

%matplotlib inline

In [ ]:
df = pd.read_csv("data/bike-station-207.csv", low_memory=False)

df.shape

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df.info()

In [ ]:
df["대여연월"] = df["대여일시"].apply(lambda x : x[:7])
df["반납연월"] = df["반납일시"].apply(lambda x : x[:7])

In [ ]:
# object 타입에는 .dt accessor를 사용할 수 없기 때문에
# 대여일시와 반납일시를 datetime 형태로 변환해 줍니다.
df["대여일시"] = pd.to_datetime(df["대여일시"])
df["반납일시"] = pd.to_datetime(df["반납일시"])

df[["대여일시", "반납일시"]].dtypes

In [ ]:
df["대여연도"] = df["대여일시"].dt.year
df["대여월"] = df["대여일시"].dt.month
df["대여일"] = df["대여일시"].dt.day
df["대여시간"] = df["대여일시"].dt.hour
df["대여요일"] = df["대여일시"].dt.dayofweek
df["대여일자"] = df["대여일시"].dt.date

df.sample()

In [ ]:
df["반납연도"] = df["반납일시"].dt.year
df["반납월"] = df["반납일시"].dt.month
df["반납일"] = df["반납일시"].dt.day
df["반납시간"] = df["반납일시"].dt.hour
df["반납요일"] = df["반납일시"].dt.dayofweek
df["반납일자"] = df["반납일시"].dt.date

df.sample()

In [ ]:
# 수치를 집계해보기 전에 countplot으로 분석해 본다.
plt.figure(figsize=(15, 5))
sns.countplot(data=df, x="대여시간")

In [ ]:
# 어느 요일에 자전거를 더 많이 타는지 비교해 본다.
plt.figure(figsize=(15, 5))
sns.countplot(data=df, x="대여요일")

In [ ]:
# 연도별 데이터를 보면 모든 월 데이터가 있지 않다. 따라서 count 값으로 시각화를 하는 것은 적절하지 않다.(평균이나 다른 수치로 보도록 한다.)
plt.figure(figsize=(15, 5))
sns.countplot(data=df, x="대여월")

### 시간대별 대여수량을 집계

In [ ]:
# value 값은 "대여일시"로 넣어주었는데 어떤 컬럼을 넣어주어도 count값을 동일하게 구한다.
# 대여일자로 구하게 되면 reset_index()에서 컬럼명이 중복되기 때문에 오류가 발생해서 다른 컬럼으로 구해왔다.
df_rent_group = df.groupby(["대여일자", "대여연월", "대여연도", "대여월", "대여일", "대여요일"])["대여일시"].count()
df_rent_group.head()

In [ ]:
df_rent = pd.DataFrame(df_rent_group).reset_index()
df_rent.columns = ["대여일자", "대여연월", "대여연도", 
                   "대여월", "대여일", "대여요일", "대여수"]
df_rent.head()

In [ ]:
# 수치 데이터를 히스토그램으로 표현해 본다.
# 수치 데이터를 막대그래프로 표현하기 위해서는 도수분포표를 만들고 이를 시각화 하는 것이 히스토그램이다.
df.hist(figsize = (15,15), bins=25)

In [ ]:
df_rent.groupby('대여요일')['대여수'].mean().plot.bar(
    title="요일별 평균 자전거 대여수", rot=0)

In [ ]:
df_rent.groupby('대여연도')['대여수'].mean().plot.bar(rot=0)

In [ ]:
df_rent.groupby('대여일자')['대여수'].mean().plot(rot=30, figsize=(15, 5))

In [ ]:
df_rent_2017 = df_rent[df_rent["대여연도"] == 2017]
df_rent_2017.groupby('대여월')['대여수'].mean().plot.bar(rot=0)

In [ ]:
df_rent.groupby('대여월')['대여수'].mean().plot.bar(rot=0)

In [ ]:
# 대여연도와 월별 대여수 평균을 구한다.
df_rent.groupby('대여연월')['대여수'].mean()

In [ ]:
df_rent.groupby('대여연월')['대여수'].mean().plot.bar(rot=30, figsize=(15, 5))

In [ ]:
df_rent.groupby('대여연월')['대여수'].mean().plot(rot=30, figsize=(15, 5))

In [ ]:
fig,axs = plt.subplots(3,1)

df_rent["대여일"].plot(figsize = (15,8), title = "일별", ax = axs[0])
df_rent["대여요일"].plot(figsize = (15,8), title = "요일별", ax = axs[1])
df_rent["대여월"].plot(figsize = (15,8), title = "월별", ax = axs[2])

In [ ]:
df_rent_day = df_rent.groupby(["대여일자"])["대여수"].sum()
df_rent_day.head()

## Simple Moving Average

In [ ]:
# Determine rolling statistics
# 30일치 rolling mean을 구해서 시각화 합니다.
rolmean = df_rent_day.rolling(window=30).mean() 
# window size 30 denotes 30 days, giving rolling mean at monthly level
rolstd = df_rent_day.rolling(window=30).std()

In [ ]:
df_rent_day.plot(label='Original')
rolmean.plot(label='Rolling Mean')
rolstd.plot(label='Rolling Std', figsize=(15, 5))

plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')

## Weighted Moving Average
* exponentially-weighted-windows
* [Computational tools — pandas 0.25.0 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#exponentially-weighted-windows)

In [ ]:
df_rent['WMA12'] = df_rent['대여수'].ewm(span=12).mean()
df_rent['WMA6'] = df_rent['대여수'].ewm(span=6).mean()

In [ ]:
df_rent[['대여일자', '대여수', 'WMA6','WMA12']].plot(x='대여일자', figsize=(15,5))

## Simple Exponential Smoothing

In [ ]:
# 데이터셋을 train과 test 로 나눈다.
train = pd.DataFrame(df_rent_day[:800])
test = pd.DataFrame(df_rent_day[800:])

In [ ]:
train["대여수"].plot(figsize=(15,5))
test["대여수"].plot()

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing
ses_model = SimpleExpSmoothing(pd.np.asarray(train['대여수'].astype(np.float)))

In [ ]:
ses_result = ses_model.fit()
ses_result

In [ ]:
y_hat = test.copy()
y_hat['SES'] = ses_result.forecast(len(test))
y_hat['SES'][:3]

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train['대여수'], label='Train')
plt.plot(test['대여수'], label='Test')
plt.plot(y_hat['SES'], label='Simple Exp Smoothing')
plt.legend()

In [ ]:
rmse = {}
# root mean squared error 로 오차를 계산해 본다.
rmse["SES"] = np.sqrt(np.square(test['대여수'] - y_hat['SES']).mean())
rmse["SES"]

## Expanding

In [ ]:
# Expanding
df_rent['대여수'].expanding(min_periods=1).mean().plot(figsize=(15,5))

### Dickey–Fuller test

In [ ]:
# 결과의 p-value 가 5%를 벗어나기 때문에 non-stationary 데이터라고 볼 수 있다.
# AIC - 회귀에서 예측변수(predictor)를 고를 때 사용하며, 아카이케(Akaike)의 정보 기준(AIC; Akaike’s information Criterion)
# AIC 출처 : [8.6 추정과 차수 선택 | Forecasting: Principles and Practice](https://otexts.com/fppkr/arima-estimation.html)
from statsmodels.tsa.stattools import adfuller
# Perform Dickey–Fuller test:
print('Results of Dickey Fuller Test:')
dftest = adfuller(df_rent['대여수'], autolag='AIC')
dfoutput = pd.Series(
    dftest[0:4], 
    index=['Test Statistic',
           'p-value',
           '#Lags Used',
           'Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

## Time Series data Decomposition(시계열 데이터 분해)
* Seasonal
* Trend
* Residual

## Additive Model

* 값을 더해서 구한다.

$y_t = Level + Trend + Seasonality + Noise$


In [ ]:
from statsmodels.api import tsa

# Additive model
res = tsa.seasonal_decompose(df_rent["대여수"], freq=30, model="additive")
fig = res.plot()

## Multiplicative Model
*  값을 곱해서 구한다.

$y_t = Level \times Trend \times Seasonality \times Noise$

In [ ]:
# multiplicative
res = tsa.seasonal_decompose(df_rent["대여수"], freq=7, model="multiplicative")
fig = res.plot()

## ACF, PACF
* ARIMA 모델 사용시 lag 값에 따른 절단값으로 p,d,q값을 찾기 위해 그려본다.
* AUTO.ARIMA 함수를 사용하여 최적화된 파라미터를 찾을 수도 있다.

## ACF : 자기상관 함수 AutoCorrelation Function

## PACF : 부분 자기상관 함수 Partial AutoCorrelation Function 

* p – Lag value where the PACF chart crosses the upper confidence interval for the first time.
* q – Lag value where the ACF chart crosses the upper confidence interval for the first time.

* 참고 : [Detecting stationarity in time series data - Towards Data Science](https://towardsdatascience.com/detecting-stationarity-in-time-series-data-d29e0a21e638)

In [ ]:
from statsmodels.graphics import tsaplots
# lag는 0부터 설정할 수 있으나 너무 낮으면  그래프를 보기 어렵다.
# 0은 제외하고 본다.

ax1 = plt.subplot(211)
tsaplots.plot_acf(df_rent["대여수"], lags=30, ax=ax1)
ax2 = plt.subplot(212)
tsaplots.plot_pacf(df_rent["대여수"], lags=30, ax=ax2)
plt.tight_layout()

## Seasonal ARIMA
* 참고 : [An End-to-End Project on Time Series Analysis and Forecasting with Python](https://towardsdatascience.com/an-end-to-end-project-on-time-series-analysis-and-forecasting-with-python-4835e6bf050b)

### Trend
* p: Trend autoregression order. AR(p)모형의 p차수
* d: Trend difference order. 트랜드를 제거하여 안정시계열로 만들기 위한 I(d)의 차분 차수 d
* q: Trend moving average order. MA(q)의 q차수 

### Seasonal
* P: Seasonal autoregressive order.
* D: Seasonal difference order.
* Q: Seasonal moving average order.
* m: The number of time steps for a single seasonal period.

In [ ]:
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
print(pdq)
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]
seasonal_pdq

In [ ]:
# Grid Search와 유사하게 최적의 파라메터 값을 찾는다.
y = train['대여수'].to_list()

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = tsa.statespace.SARIMAX(y,
                                        order=param,
                                        seasonal_order=param_seasonal,
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}7 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
# ARIMA(p,d,q)(P,D,Q)m
# ARIMA(1, 1, 1)x(0, 1, 1, 7)7 - AIC:10083.141308943608
arima = tsa.statespace.SARIMAX(train['대여수'].to_list(),
                                  order=(1,1,1),
                                  seasonal_order=(0,1,1,7),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)
# 학습
arima_result = arima.fit()
print(arima_result.summary().tables[1])

In [ ]:
arima_result.plot_diagnostics(figsize=(15, 10))

In [ ]:
# 예측
predict_value = arima_result.predict(start=801, end=880, dynamic=True)
predict_value[:5]

In [ ]:
y_hat['ARIMA'] = predict_value
y_hat['ARIMA'].head()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train['대여수'], label='Train')
plt.plot(test['대여수'], label='Test')
plt.plot(y_hat['ARIMA'], label='Seasonal ARIMA')
plt.legend()

In [ ]:
# root mean squared error 로 오차를 계산해 본다.
rmse["ARIMA"] = np.sqrt(np.square(test['대여수'] - y_hat['ARIMA']).mean())
rmse["ARIMA"]

In [ ]:
rmse

## fbprophet
* https://anaconda.org/conda-forge/fbprophet
* conda install -c conda-forge fbprophet
* pip로 설치한다면 pystan이 설치되어 있어야 함

In [ ]:
# !pip install pystan
# !pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
p_train = train.reset_index().copy()
p_train.columns = ["ds", "y"]
p_train["y"] = np.log(p_train["y"])
p_train.head()

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(p_train)

In [ ]:
future = m.make_future_dataframe(periods=80)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
# plotly 가 설치되어 있지 않다면 아래의 명령어로 설치가 필요하다.
# 아나콘다에 설치 시 : conda install -c plotly plotly 
from fbprophet.plot import plot_plotly
import plotly.offline as py

py.init_notebook_mode()

fig = plot_plotly(m, forecast)  # This returns a plotly Figure
py.iplot(fig)